# QoS, to make Networks Dependable
<sup> <b>Author:</b> Sarah Samuel (sasamuel@cisco.com) <b>Inputs:</b> Vineet Upendra (vupendra@cisco.com) </sup>

## What is Quality of Service?

Quality of Service (QoS) is the technique of prioritizing traffic flows and providing preferential forwarding for higher-priority packets. The fundamental reason to implement QoS in your network is to provide guaranteed end-to-end traffic forwarding.

Modular QoS provides features that ensure better and more predictable network service by:
* supporting bandwidth allocation
* improving loss characteristics
* avoiding and managing network congestion
* metering network traffic
* setting traffic flow priorities across the network.

This notebook shows you how you can configure QoS on the network with various examples.

NETWORK TOPOLOGY:
![Topology](./images/QoS.png)

## Before You Begin

Before you begin to configure QoS on the simulator, ensure to complete the following prerequisites.
* [Set up simulator](#step1)
* [Add the base configuration](#step2)
* [Design the virtual router topology](#step3)
* [Deploy the configurations](#step4)


## QoS Configurations

After the simulated topology is up and running with the base configurations, we apply the QoS configurations in the below sections:
* [Create a class-map](#step5)
* [Create a policy-map](#step6)
* [Attach the traffic policy to an interface](#step7)
* [Verification of the traffic policy](#step8)
* [Example 1: IP Precedence Marking Configuration](#step9)
* [Example 2: IP DSCP Marking Configuration](#step21)
* [Example 3: QoS Group Marking Configuration](#step10)
* [Example 4: MPLS Experimental Bit Imposition Marking Configuration](#step12)
* [Example 5: MPLS Experimental Topmost Marking Configuration](#step13)

### <a name="step1"></a>Set up Simulator


>First, you'll have to set up the simulator to build your network. The following sets up our python enviroment and the simulator object.

In [1]:
from lib.xr import *
nodes = {
         'PE1':'', 
         'P1':'',
         'P2':'', 
         'PE2':'', 
         'trex':''
        }

tb = access_device_consoles("lib/tb.yaml", nodes)

3.6.9 (default, Jun 29 2022, 11:45:57) 
[GCC 8.4.0]

*** Logging into the devices ***

2022-09-21 05:17:10,211: %UNICON-INFO: +++ rp2 logfile /tmp/rp2-cli-20220921T051710209.log +++

2022-09-21 05:17:10,212: %UNICON-INFO: +++ Unicon plugin iosxr/spitfire (unicon.plugins.iosxr.spitfire) +++
Password: 

2022-09-21 05:17:10,363: %UNICON-INFO: +++ connection to spawn: ssh -l cisco 10.10.20.227 -p 63506, id: 140531386314536 +++

2022-09-21 05:17:10,366: %UNICON-INFO: connection to rp2

Last login: Wed Sep 21 04:41:50 2022 from 192.168.122.1



RP/0/RP0/CPU0:P2#

2022-09-21 05:17:10,896: %UNICON-INFO: Learned hostname(s): 'P2'.

2022-09-21 05:17:10,898: %UNICON-INFO: +++ initializing handle +++

2022-09-21 05:17:10,976: %UNICON-INFO: +++ P2 with via 'cli': executing command 'terminal length 0' +++
terminal length 0
Wed Sep 21 05:13:17.347 UTC
RP/0/RP0/CPU0:P2#

2022-09-21 05:17:11,270: %UNICON-INFO: +++ P2 with via 'cli': executing command 'terminal width 0' +++
terminal width 0
Wed Sep 21 0

### <a name="step2"></a>Add Base Router Configuration

>This defines and starts up a small two router topology for our lab and a Linux server to use on the same LAN, with SSH to all of the routers.
Takes a minute or five to start. 

In [2]:
out = nodes['PE1'].configure(pe1_config_str)
out = nodes['P1'].configure(p1_config_str)
out = nodes['P2'].configure(p2_config_str)
out = nodes['PE2'].configure(pe2_config_str)


2022-09-21 05:17:27,197: %UNICON-INFO: +++ P2 with via 'cli': configure +++
configure terminal
Wed Sep 21 05:13:33.629 UTC
RP/0/RP0/CPU0:P2(config)#
RP/0/RP0/CPU0:P2(config)#ssh server v2
RP/0/RP0/CPU0:P2(config)#ssh server netconf
RP/0/RP0/CPU0:P2(config)#netconf-yang agent ssh
RP/0/RP0/CPU0:P2(config)#!
RP/0/RP0/CPU0:P2(config)#hostname P2
RP/0/RP0/CPU0:P2(config)#!
RP/0/RP0/CPU0:P2(config)#interface Loopback0
RP/0/RP0/CPU0:P2(config-if)# ipv4 address 10.200.200.202 255.255.255.255
RP/0/RP0/CPU0:P2(config-if)#!
RP/0/RP0/CPU0:P2(config-if)#interface FourHundredGigE0/0/0/0
RP/0/RP0/CPU0:P2(config-if)# description Connected_to_P1
RP/0/RP0/CPU0:P2(config-if)# mtu 9216
RP/0/RP0/CPU0:P2(config-if)# ipv4 address 10.120.3.2 255.255.255.0
RP/0/RP0/CPU0:P2(config-if)# no shutdown
RP/0/RP0/CPU0:P2(config-if)#!
RP/0/RP0/CPU0:P2(config-if)#interface FourHundredGigE0/0/0/1
RP/0/RP0/CPU0:P2(config-if)# description Connected_to_PE2
RP/0/RP0/CPU0:P2(config-if)# mtu 9216
RP/0/RP0/CPU0:P2(config-if)# 

### Start the network traffic

In [3]:
# Import the function that generates high priority traffic
from traffic.TrafficGenerator import generate_hipriority_traffic
trex_ipaddress = str(nodes['trex'].connections.cli.ip)
trex_port = str(nodes['trex'].connections.cli.port)
client1, client2, interact1, interact2 = generate_hipriority_traffic (trex_ipaddress, trex_port)

Last login: Tue Sep 20 09:15:27 2022 from sonic-6
[root@localhost ~]# ifconfig eth1 10.0.0.1 netmask 255.255.255.0 up; ifconfig et h2 10.1.1.1 netmask 255.255.255.0 up
[root@localhost ~]# cd /opt/cisco/trex/latest/
[root@localhost latest]# ./t-rex-64 -i
Starting Scapy server........ Scapy server is started
Trying to bind to vfio-pci ...
Trying to compile and bind to igb_uio ...
ERROR: We don't have precompiled igb_uio.ko module for your kernel version.
Will try compiling automatically...
Success.

/usr/bin/python3 dpdk_nic_bind.py --bind=igb_uio 0000:00:04.0 0000:00:05.0 
The ports are bound/configured.
Starting  TRex v2.99 please wait  ... 
 set driver name net_virtio 
 driver capability  : SLRO 
 set dpdk queues mode to ONE_QUE 
 Number of ports found: 2
zmq publisher at: tcp://*:4500
 wait 1 sec Last login: Tue Sep 20 09:15:53 2022 from sonic-6
[root@localhost ~]# cd /opt/cisco/trex/latest/
[root@localhost latest]# ./trex-console
start -f stl/dscp_traffic1.py -d 1h -m 100pps -p 0

U

## <a name="step5"></a>Create a class map

>First, create the class-map by executing the below steps.

In [4]:
# The below steps configures the class-map.
out = nodes['PE1'].configure('''
class-map match-any class1
 match dscp ef
 end-class-map
''')


2022-09-20 09:19:25,100: %UNICON-INFO: +++ PE1 with via 'cli': configure +++
configure terminal
Tue Sep 20 09:15:27.945 UTC
RP/0/RP0/CPU0:PE1(config)#
RP/0/RP0/CPU0:PE1(config)#class-map match-any class1
RP/0/RP0/CPU0:PE1(config-cmap)# match dscp ef
RP/0/RP0/CPU0:PE1(config-cmap)# end-class-map
RP/0/RP0/CPU0:PE1(config)#commit
Tue Sep 20 09:15:28.433 UTC
RP/0/RP0/CPU0:PE1(config)#end
RP/0/RP0/CPU0:PE1#


>Verify the class-map with show commands.

In [5]:
out = nodes['PE1'].execute('show class-map class1')


2022-09-20 09:20:12,942: %UNICON-INFO: +++ PE1 with via 'cli': executing command 'show class-map class1' +++
show class-map class1
Tue Sep 20 09:16:15.723 UTC
1) ClassMap: class1    Type: qos 
    Referenced by 0 Policymaps 

RP/0/RP0/CPU0:PE1#


## <a name="step6"></a>Create a policy-map

In [7]:
# This step configures the policy-map using the class-map created in the previous step.
out = nodes['PE1'].configure('''
policy-map policy1
 class class1
  set qos-group 4
 class class-default 
 end-policy-map
''')


2022-09-20 09:20:47,017: %UNICON-INFO: +++ PE1 with via 'cli': configure +++
configure terminal
Tue Sep 20 09:16:49.862 UTC
RP/0/RP0/CPU0:PE1(config)#
RP/0/RP0/CPU0:PE1(config)#policy-map policy1
RP/0/RP0/CPU0:PE1(config-pmap)# class class1
RP/0/RP0/CPU0:PE1(config-pmap-c)#  set qos-group 4
RP/0/RP0/CPU0:PE1(config-pmap-c)# class class-default 
RP/0/RP0/CPU0:PE1(config-pmap-c)# end-policy-map
RP/0/RP0/CPU0:PE1(config)#commit
Tue Sep 20 09:16:50.434 UTC
RP/0/RP0/CPU0:PE1(config)#end
RP/0/RP0/CPU0:PE1#


> Verify the policy-map with show commands.

In [8]:
out = nodes['PE1'].execute('show policy-map policy1')


2022-09-20 09:20:50,513: %UNICON-INFO: +++ PE1 with via 'cli': executing command 'show policy-map policy1' +++
show policy-map policy1
                                   ^
% Invalid input detected at '^' marker.
RP/0/RP0/CPU0:PE1#


## <a name="step7"></a>Apply the traffic policy to an interface

In [9]:
# Applying traffic policy to an interface in ingress direction.
# Note that policy can be applied on the interface in the ingress and egress direction.
out = nodes['PE1'].configure('''
interface FourHundredGigE0/0/0/0
 service-policy input policy1
''')


2022-09-20 09:21:18,988: %UNICON-INFO: +++ PE1 with via 'cli': configure +++
configure terminal
Tue Sep 20 09:17:21.830 UTC
RP/0/RP0/CPU0:PE1(config)#
RP/0/RP0/CPU0:PE1(config)#interface FourHundredGigE0/0/0/0
RP/0/RP0/CPU0:PE1(config-if)# service-policy input policy1
RP/0/RP0/CPU0:PE1(config-if)#commit
Tue Sep 20 09:17:22.243 UTC
RP/0/RP0/CPU0:PE1(config-if)#end
RP/0/RP0/CPU0:PE1#


## <a name="step8"></a>Verification of the traffic policy

In [10]:
# now display the policy applied on the interface.
out = nodes['PE1'].execute('show qos interface FourHundredGigE0/0/0/0 input')


2022-09-20 09:21:23,141: %UNICON-INFO: +++ PE1 with via 'cli': executing command 'show qos interface FourHundredGigE0/0/0/0 input' +++
show qos interface FourHundredGigE0/0/0/0 input
Tue Sep 20 09:17:25.917 UTC
NOTE:- Configured values are displayed within parentheses
Interface FourHundredGigE0/0/0/0 ifh 0xf000138  -- input policy
NPU Id:                        0
Total number of classes:       2
Interface Bandwidth:           400000000 kbps
Policy Name:                   policy1
Accounting Type:               Layer1 (Include Layer 1 encapsulation and above)
------------------------------------------------------------------------------
Level1 Class                             =   class1
New qos group                            =   4
Policer not configured for this class

Level1 Class                             =   class-default
Policer not configured for this class
RP/0/RP0/CPU0:PE1#


In [11]:
# Check the traffic on the port
out = nodes['PE1'].execute('show interface FourHundredGigE0/0/0/0 accounting')
# Policy-map counter updates are v slow. To see the counters incrementing, replay this cell a few times 
out = nodes['PE1'].execute('show policy-map interface FourHundredGigE0/0/0/0 input')


2022-09-20 09:21:31,602: %UNICON-INFO: +++ PE1 with via 'cli': executing command 'show interface FourHundredGigE0/0/0/0 accounting' +++
show interface FourHundredGigE0/0/0/0 accounting
Tue Sep 20 09:17:34.373 UTC
FourHundredGigE0/0/0/0
  Protocol              Pkts In         Chars In     Pkts Out        Chars Out
  IPV4_UNICAST            16805           773030            0                0
  ARP                         4              240            2               84


RP/0/RP0/CPU0:PE1#

2022-09-20 09:21:31,911: %UNICON-INFO: +++ PE1 with via 'cli': executing command 'show policy-map interface FourHundredGigE0/0/0/0 input' +++
show policy-map interface FourHundredGigE0/0/0/0 input
Tue Sep 20 09:17:34.685 UTC

FourHundredGigE0/0/0/0 input: policy1

Class class1
  Classification statistics          (packets/bytes)     (rate - kbps)
    Matched             :                   0/0                    0
    Transmitted         :                   0/0                    0
    Total Dropped

## <a name="step9"></a>Example 1: IP Precedence Marking Configuration

> In the previous sections, you have already configured an input service-policy on the input interface **FourHundredGigE0/0/0/0** to set the qos-group value to 4 for all incoming traffic packets which have the **DSCP** bits set to **EF** (Expedited Forwarding). 

> Play the next cell to configure an output service policy named **out-policy1** on the output interface **FourHundredGigE0/0/0/1**. This service-policy matches all the outgoing trafic packets that have qos-group value set to 4 and sets the IP precedence bit to 1 for the matched packets. 

In [12]:
out = nodes['PE1'].configure('''
class-map match-any out-class1
 match qos-group 4
 end-class-map

policy-map out-policy1
 class out-class1
  set precedence 1

interface FourHundredGigE0/0/0/1
 service-policy output out-policy1
''')


2022-09-20 09:21:57,571: %UNICON-INFO: +++ PE1 with via 'cli': configure +++
configure terminal
Tue Sep 20 09:18:00.426 UTC
RP/0/RP0/CPU0:PE1(config)#
RP/0/RP0/CPU0:PE1(config)#class-map match-any out-class1
RP/0/RP0/CPU0:PE1(config-cmap)# match qos-group 4
RP/0/RP0/CPU0:PE1(config-cmap)# end-class-map
RP/0/RP0/CPU0:PE1(config)#
RP/0/RP0/CPU0:PE1(config)#policy-map out-policy1
RP/0/RP0/CPU0:PE1(config-pmap)# class out-class1
RP/0/RP0/CPU0:PE1(config-pmap-c)#  set precedence 1
RP/0/RP0/CPU0:PE1(config-pmap-c)#
RP/0/RP0/CPU0:PE1(config-pmap-c)#interface FourHundredGigE0/0/0/1
RP/0/RP0/CPU0:PE1(config-if)# service-policy output out-policy1
RP/0/RP0/CPU0:PE1(config-if)#commit
Tue Sep 20 09:18:01.248 UTC
RP/0/RP0/CPU0:PE1(config-if)#end
RP/0/RP0/CPU0:PE1#


> The egress packets on interface **FourHundredGigE0/0/0/1**  matches the class-map **out-class1** in the service-policy **out-policy1**. You can view the policy-map details and counters by playing the next cell. Note that the policy-map counter updates are slow. To see the policy-map counters incrementing, replay this cell a few times

In [13]:
# now display the policy applied on the interface.
out = nodes['PE1'].execute('show qos interface FourHundredGigE0/0/0/1 output')

# Other useful show commands
out = nodes['PE1'].execute('show interface FourHundredGigE0/0/0/1 accounting')
# Policy-map counter updates are v slow. To see the counters incrementing, replay this cell a few times 
out = nodes['PE1'].execute('show policy-map interface FourHundredGigE0/0/0/1 output')



2022-09-20 09:22:01,766: %UNICON-INFO: +++ PE1 with via 'cli': executing command 'show qos interface FourHundredGigE0/0/0/1 output' +++
show qos interface FourHundredGigE0/0/0/1 output
Tue Sep 20 09:18:04.540 UTC
NOTE:- Configured values are displayed within parentheses
Interface FourHundredGigE0/0/0/1 ifh 0xf0001a0  -- output policy
NPU Id:                        0
Total number of classes:       2
Interface Bandwidth:           400000000 kbps
Policy Name:                   out-policy1
VOQ Base:                      0
Accounting Type:               Layer1 (Include Layer 1 encapsulation and above)
VOQ Mode:                      8
Shared Counter Mode:           1
------------------------------------------------------------------------------
Level1 Class                             =   out-class1
New prec                                 =   1

Level1 Class                             =   class-default
RP/0/RP0/CPU0:PE1#

2022-09-20 09:22:02,040: %UNICON-INFO: +++ PE1 with via 'cli': exec

In [14]:
# Remove the previous commit in order to configure more policies.
out = nodes['PE1'].execute('rollback configuration last 1')


2022-09-20 09:22:32,979: %UNICON-INFO: +++ PE1 with via 'cli': executing command 'rollback configuration last 1' +++
rollback configuration last 1
Tue Sep 20 09:18:35.761 UTC

Loading Rollback Changes.
Loaded Rollback Changes in 1 sec 
Committing.
5 items committed in 1 sec (4)items/sec
Updating.
Updated Commit database in 1 sec 
Configuration successfully rolled back 1 commits.
RP/0/RP0/CPU0:PE1#


## <a name="step21"></a>Example 2: IP DSCP Marking Configuration

> Play the next cell to configure an output service policy named **policy-dscp** on the output interface **FourHundredGigE0/0/0/1**. This service-policy matches all the outgoing trafic packets that have qos-group value set to 4 and sets the IP DSCP value to 5 for the matched packets. 

In [15]:
out = nodes['PE1'].configure('''
class-map match-any out-class1
 match qos-group 4
 end-class-map

policy-map policy-dscp
 class out-class1
  set dscp 5
 class class-default
 end-policy-map

interface FourHundredGigE0/0/0/1
 service-policy output policy-dscp
''')


2022-09-20 09:22:40,148: %UNICON-INFO: +++ PE1 with via 'cli': configure +++
configure terminal
Tue Sep 20 09:18:42.989 UTC
RP/0/RP0/CPU0:PE1(config)#
RP/0/RP0/CPU0:PE1(config)#class-map match-any out-class1
RP/0/RP0/CPU0:PE1(config-cmap)# match qos-group 4
RP/0/RP0/CPU0:PE1(config-cmap)# end-class-map
RP/0/RP0/CPU0:PE1(config)#
RP/0/RP0/CPU0:PE1(config)#policy-map policy-dscp
RP/0/RP0/CPU0:PE1(config-pmap)# class out-class1
RP/0/RP0/CPU0:PE1(config-pmap-c)#  set dscp 5
RP/0/RP0/CPU0:PE1(config-pmap-c)# class class-default
RP/0/RP0/CPU0:PE1(config-pmap-c)# end-policy-map
RP/0/RP0/CPU0:PE1(config)#
RP/0/RP0/CPU0:PE1(config)#interface FourHundredGigE0/0/0/1
RP/0/RP0/CPU0:PE1(config-if)# service-policy output policy-dscp
RP/0/RP0/CPU0:PE1(config-if)#commit
Tue Sep 20 09:18:43.980 UTC
RP/0/RP0/CPU0:PE1(config-if)#end
RP/0/RP0/CPU0:PE1#


> The egress traffic packets on interface **FourHundredGigE0/0/0/1**  matches the class-map **out-class1** in the service-policy **policy-dscp**, because the input service-policy on the input interface ie **FourHundredGigE0/0/0/0** sets the qos-group value to 4 for the incoming packets.

In [16]:
# now display the policy applied on the interface.
out = nodes['PE1'].execute('show qos interface FourHundredGigE0/0/0/1 output')

# Other useful show commands
out = nodes['PE1'].execute('show interface FourHundredGigE0/0/0/1 accounting')
# Policy-map counter updates are v slow. To see the counters incrementing, replay this cell a few times 
out = nodes['PE1'].execute('show policy-map interface FourHundredGigE0/0/0/1 output')


2022-09-20 09:23:22,958: %UNICON-INFO: +++ PE1 with via 'cli': executing command 'show qos interface FourHundredGigE0/0/0/1 output' +++
show qos interface FourHundredGigE0/0/0/1 output
Tue Sep 20 09:19:25.730 UTC
NOTE:- Configured values are displayed within parentheses
Interface FourHundredGigE0/0/0/1 ifh 0xf0001a0  -- output policy
NPU Id:                        0
Total number of classes:       2
Interface Bandwidth:           400000000 kbps
Policy Name:                   policy-dscp
VOQ Base:                      0
Accounting Type:               Layer1 (Include Layer 1 encapsulation and above)
VOQ Mode:                      8
Shared Counter Mode:           1
------------------------------------------------------------------------------
Level1 Class                             =   out-class1
New dscp                                 =   5

Level1 Class                             =   class-default
RP/0/RP0/CPU0:PE1#

2022-09-20 09:23:23,220: %UNICON-INFO: +++ PE1 with via 'cli': exec

In [17]:
# Remove the previous commit in order to configure more policies.
out = nodes['PE1'].execute('rollback configuration last 1')


2022-09-20 09:23:38,130: %UNICON-INFO: +++ PE1 with via 'cli': executing command 'rollback configuration last 1' +++
rollback configuration last 1
Tue Sep 20 09:19:40.907 UTC

Loading Rollback Changes.
Loaded Rollback Changes in 1 sec 
Committing.
5 items committed in 1 sec (4)items/sec
Updating.
Updated Commit database in 1 sec 
Configuration successfully rolled back 1 commits.
RP/0/RP0/CPU0:PE1#


## <a name="step10"></a>Example 3: QoS Group Marking Configuration

> In this example, you will unconfigure the input service-policy **policy1** and then configure another input service policy named **policy-qg** on the interface **FourHundredGigE0/0/0/0**. The qos-group value is set to 1 for all incoming traffic packets that match **class3**. 

In [38]:
out = nodes['PE1'].configure('''
class-map match-any class3
 match protocol ipv4
 match access-group ipv4 101
 end-class-map

policy-map policy-qg
 class class3
  set qos-group 1
  end-policy-map

interface FourHundredGigE0/0/0/0
 no service-policy input policy1
 service-policy input policy-qg
''')


2021-06-17 13:43:16,631: %UNICON-INFO: +++ R1 with alias 'cli': configure +++
configure terminal
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#class-map match-any class3
RP/0/RP0/CPU0:R1(config-cmap)# match protocol ipv4
RP/0/RP0/CPU0:R1(config-cmap)# match access-group ipv4 101
RP/0/RP0/CPU0:R1(config-cmap)# end-class-map
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#policy-map policy-qg
RP/0/RP0/CPU0:R1(config-pmap)# class class3
RP/0/RP0/CPU0:R1(config-pmap-c)#  set qos-group 1
RP/0/RP0/CPU0:R1(config-pmap-c)#  end-policy-map
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#interface FourHundredGigE0/0/0/0
RP/0/RP0/CPU0:R1(config-if)# no service-policy input policy1
RP/0/RP0/CPU0:R1(config-if)# service-policy input policy-qg
RP/0/RP0/CPU0:R1(config-if)#commit
RP/0/RP0/CPU0:R1(config-if)#end
RP/0/RP0/CPU0:R1#


In [39]:
# now Display the policy applied on the interface.
out = nodes['PE1'].execute('show qos interface FourHundredGigE0/0/0/0 input')

# Other useful show commands
out = nodes['PE1'].execute('show interface FourHundredGigE0/0/0/0 accounting')
# Policy-map counter updates are v slow. To see the counters incrementing, replay this cell a few times 
out = nodes['PE1'].execute('show policy-map interface FourHundredGigE0/0/0/0 input')


2021-06-17 13:44:19,438: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'show qos interface FourHundredGigE0/0/0/0 input' +++
show qos interface FourHundredGigE0/0/0/0 input
NOTE:- Configured values are displayed within parentheses
Interface FourHundredGigE0/0/0/0 ifh 0xf000138  -- input policy
NPU Id:                        0
Total number of classes:       2
Interface Bandwidth:           400000000 kbps
Policy Name:                   policy-qg
Accounting Type:               Layer1 (Include Layer 1 encapsulation and above)
------------------------------------------------------------------------------
Level1 Class                             =   class3
New qos group                            =   1
Policer not configured for this class

Level1 Class                             =   class-default
Policer not configured for this class
RP/0/RP0/CPU0:R1#

2021-06-17 13:44:19,675: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'show interface FourHundredGigE0/0/0/0 accoun

> Unconfigure the service-policy using the "no" form of the service-policy command instead of **rollback configuration**.

In [40]:
# Remove the policy-map using "no" of the command
out = nodes['PE1'].configure('''
interface FourHundredGigE0/0/0/0
no service-policy input policy-qg
''')


2021-06-17 13:45:16,297: %UNICON-INFO: +++ R1 with alias 'cli': configure +++
configure terminal
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#interface FourHundredGigE0/0/0/0
RP/0/RP0/CPU0:R1(config-if)#no service-policy input policy-qg
RP/0/RP0/CPU0:R1(config-if)#commit
RP/0/RP0/CPU0:R1(config-if)#end
RP/0/RP0/CPU0:R1#


## <a name="step12"></a>Example 4: MPLS Experimental Bit Imposition Marking Configuration

>In this example, you will configure an input service policy **policy-mex-imp**. This service policy is associated to a class-map **class3** through the use of the class command. This service-policy sets the MPLS EXP bits of the MPLS labels to 1 for the matched incoming packets.

In [41]:
out = nodes['PE1'].configure('''
class-map match-any class3
 match protocol ipv4
 match access-group ipv4 101
 end-class-map

policy-map policy-mex-imp
 class class3
  set mpls exp imposition 1
  end-policy-map

interface FourHundredGigE0/0/0/0
 service-policy input policy-mex-imp
''')


2021-06-17 13:46:04,818: %UNICON-INFO: +++ R1 with alias 'cli': configure +++
configure terminal
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#class-map match-any class3
RP/0/RP0/CPU0:R1(config-cmap)# match protocol ipv4
RP/0/RP0/CPU0:R1(config-cmap)# match access-group ipv4 101
RP/0/RP0/CPU0:R1(config-cmap)# end-class-map
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#policy-map policy-mex-imp
RP/0/RP0/CPU0:R1(config-pmap)# class class3
RP/0/RP0/CPU0:R1(config-pmap-c)#  set mpls exp imposition 1
RP/0/RP0/CPU0:R1(config-pmap-c)#  end-policy-map
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#interface FourHundredGigE0/0/0/0
RP/0/RP0/CPU0:R1(config-if)# service-policy input policy-mex-imp
RP/0/RP0/CPU0:R1(config-if)#commit
RP/0/RP0/CPU0:R1(config-if)#end
RP/0/RP0/CPU0:R1#


In [45]:
# now display the policy applied on the interface.
out = nodes['PE1'].execute('show qos interface FourHundredGigE0/0/0/0 input')

# Other useful show commands
out = nodes['PE1'].execute('show interface FourHundredGigE0/0/0/0 accounting')
# Policy-map counter updates are v slow. To see the counters incrementing, replay this cell a few times 
out = nodes['PE1'].execute('show policy-map interface FourHundredGigE0/0/0/0 input')


2021-06-17 13:47:00,064: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'show qos interface FourHundredGigE0/0/0/0 input' +++
show qos interface FourHundredGigE0/0/0/0 input
NOTE:- Configured values are displayed within parentheses
Interface FourHundredGigE0/0/0/0 ifh 0xf000138  -- input policy
NPU Id:                        0
Total number of classes:       2
Interface Bandwidth:           400000000 kbps
Policy Name:                   policy-mex-imp
Accounting Type:               Layer1 (Include Layer 1 encapsulation and above)
------------------------------------------------------------------------------
Level1 Class                             =   class3
New imposition exp                       =   1
Policer not configured for this class

Level1 Class                             =   class-default
Policer not configured for this class
RP/0/RP0/CPU0:R1#

2021-06-17 13:47:00,301: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'show interface FourHundredGigE0/0/0/0 a

In [46]:
# Remove the previous commit in order to configure more policies.
out = nodes['PE1'].execute('rollback configuration last 1')


2021-06-17 13:47:05,248: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'rollback configuration last 1' +++
rollback configuration last 1

Loading Rollback Changes.
Loaded Rollback Changes in 1 sec 
Committing.
5 items committed in 1 sec (4)items/sec
Updating.
Updated Commit database in 1 sec 
Configuration successfully rolled back 1 commits.
RP/0/RP0/CPU0:R1#


## <a name="step13"></a>Example 5: MPLS Experimental Topmost Marking Configuration

> Play this cell to configure a service policy **policy-mex-top**. This service policy is associated to a class map **class-mex-top** through the use of the class command, and then the service policy is applied in the output direction on interface **FourHundredGigE0/0/0/1**. The MPLS EXP bits on the topmost label are set to 1 for all matched traffic packets.

In [47]:
out = nodes['PE1'].configure('''
class-map match-any class-mex-top
 match mpls exp topmost 1
 end-class-map

policy-map policy-mex-top
 class class-mex-top
  set mpls exp topmost 2
  end-policy-map

interface FourHundredGigE0/0/0/1
 service-policy output policy-mex-top
''')


2021-06-17 13:51:56,712: %UNICON-INFO: +++ R1 with alias 'cli': configure +++
configure terminal
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#class-map match-any class-mex-top
RP/0/RP0/CPU0:R1(config-cmap)# match mpls exp topmost 1
RP/0/RP0/CPU0:R1(config-cmap)# end-class-map
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#policy-map policy-mex-top
RP/0/RP0/CPU0:R1(config-pmap)# class class-mex-top
RP/0/RP0/CPU0:R1(config-pmap-c)#  set mpls exp topmost 2
RP/0/RP0/CPU0:R1(config-pmap-c)#  end-policy-map
RP/0/RP0/CPU0:R1(config)#
RP/0/RP0/CPU0:R1(config)#interface FourHundredGigE0/0/0/1
RP/0/RP0/CPU0:R1(config-if)# service-policy output policy-mex-top
RP/0/RP0/CPU0:R1(config-if)#commit
RP/0/RP0/CPU0:R1(config-if)#end
RP/0/RP0/CPU0:R1#


In [51]:
# now display the policy applied on the interface.
out = nodes['PE1'].execute('show qos interface FourHundredGigE0/0/0/1 output')

# Other useful show commands
out = nodes['PE1'].execute('show interface FourHundredGigE0/0/0/1 accounting')
# Policy-map counter updates are v slow. To see the counters incrementing, replay this cell a few times 
out = nodes['PE1'].execute('show policy-map interface FourHundredGigE0/0/0/1 output')


2021-06-17 13:54:14,566: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'show qos interface FourHundredGigE0/0/0/1 output' +++
show qos interface FourHundredGigE0/0/0/1 output
NOTE:- Configured values are displayed within parentheses
Interface FourHundredGigE0/0/0/1 ifh 0xf0001a0  -- output policy
NPU Id:                        0
Total number of classes:       2
Interface Bandwidth:           400000000 kbps
Policy Name:                   policy-mex-top
VOQ Base:                      0
Accounting Type:               Layer1 (Include Layer 1 encapsulation and above)
VOQ Mode:                      8
Shared Counter Mode:           1
------------------------------------------------------------------------------
Level1 Class                             =   class-mex-top
New topmost exp                          =   2

Level1 Class                             =   class-default
RP/0/RP0/CPU0:R1#

2021-06-17 13:54:14,800: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'show i

In [52]:
# Remove the previous commit in order to configure more policies.
out = nodes['PE1'].execute('rollback configuration last 1')


2021-06-17 13:54:18,968: %UNICON-INFO: +++ R1 with alias 'cli': executing command 'rollback configuration last 1' +++
rollback configuration last 1

Loading Rollback Changes.
Loaded Rollback Changes in 1 sec 
Committing.
5 items committed in 1 sec 
Updating.
Updated Commit database in 1 sec 
Configuration successfully rolled back 1 commits.
RP/0/RP0/CPU0:R1#


>Clean up configs when done.

In [3]:
out = nodes['PE1'].configure(unconfig_str)
out = nodes['P1'].configure(unconfig_str)
out = nodes['P2'].configure(unconfig_str)
out = nodes['PE2'].configure(unconfig_str)


2022-09-21 05:17:43,992: %UNICON-INFO: +++ P2 with via 'cli': configure +++
configure terminal
Wed Sep 21 05:13:50.427 UTC
RP/0/RP0/CPU0:P2(config)#
RP/0/RP0/CPU0:P2(config)#no interface Loopback0
RP/0/RP0/CPU0:P2(config)#!
RP/0/RP0/CPU0:P2(config)#no interface FourHundredGigE0/0/0/0
RP/0/RP0/CPU0:P2(config)#!
RP/0/RP0/CPU0:P2(config)#no interface FourHundredGigE0/0/0/1
RP/0/RP0/CPU0:P2(config)#!
RP/0/RP0/CPU0:P2(config)#no router ospf 10
RP/0/RP0/CPU0:P2(config)#!
RP/0/RP0/CPU0:P2(config)#no mpls ldp
RP/0/RP0/CPU0:P2(config)#!
RP/0/RP0/CPU0:P2(config)# 
RP/0/RP0/CPU0:P2(config)#commit
Wed Sep 21 05:13:51.327 UTC
RP/0/RP0/CPU0:P2(config)#end
RP/0/RP0/CPU0:P2#

2022-09-21 05:17:47,486: %UNICON-INFO: +++ PE2 with via 'cli': configure +++
configure terminal
Wed Sep 21 05:13:15.394 UTC
RP/0/RP0/CPU0:PE2(config)#
RP/0/RP0/CPU0:PE2(config)#no interface Loopback0
RP/0/RP0/CPU0:PE2(config)#!
RP/0/RP0/CPU0:PE2(config)#no interface FourHundredGigE0/0/0/0
RP/0/RP0/CPU0:PE2(config)#!
RP/0/RP0/CPU